# Pre-scraping analysis: preparing sample

In [4]:
%reset -f
import pandas as pd
import pickle
import numpy as np
import json
from tqdm import tqdm, trange
from time import sleep
import gc
import os
import morfeusz2 
import nltk
from dask import array as da
import re
import time
import itertools
#from twitter_tools import tweet_stats, preprocess_tweets
from collections import OrderedDict
from sklearn.feature_extraction.text import TfidfVectorizer
from upsetplot import plot as setplot
from tweepy import API, AppAuthHandler
from config import consumer_key, consumer_secret, access_token, access_secret
#from twitter_scrapers.FollowerScraper import FollowerScraper
from matplotlib import pyplot as plt
import datetime
import seaborn as sns
from gensim.models import KeyedVectors
from collections import defaultdict
import csv

Load network data:

In [5]:
government_followers = json.load(open("data/sample/government_followers.json", "r"))
opposition_followers = json.load(open("data/sample/opposition_followers.json", "r"))
government_superset = list(set(itertools.chain.from_iterable(government_followers.values())))
government_subsets = list(government_followers.values())
opposition_superset = list(set(itertools.chain.from_iterable(opposition_followers.values())))
opposition_subsets = list(opposition_followers.values())

In [6]:
print("Number of government followers {}. Number of opposition followers {}".format(len(government_superset), len(opposition_superset)))

Number of government followers 675773. Number of opposition followers 686198


In [14]:
gov_in_gov,gov_in_opp,opp_in_opp,opp_in_gov = pickle.load(open("data/misc/overlaps.pickle","rb"))
gov_partisans = np.array(government_superset)[(gov_in_gov > gov_in_opp)].tolist()
opp_partisans = np.array(opposition_superset)[(opp_in_opp > opp_in_gov)].tolist()

In [15]:
print("Number of government partisans {}. Number of opposition partisans {}".format(len(gov_partisans), len(opp_partisans)))

Number of government partisans 452429. Number of opposition partisans 440215


In [6]:
if os.path.isfile("data/gov_partisans.pickle") and os.path.isfile("data/opp_partisans.pickle"):
    gov_partisans = pickle.load(open("data/gov_partisans.pickle","rb"))
    opp_partisans = pickle.load(open("data/opp_partisans.pickle","rb"))
else:
    gov_partisans, opp_partisans = scraper.subsetFollowers(government_followers, opposition_followers, 6, 4)
    pickle.dump(gov_partisans,open("data/gov_partisans.pickle","wb"))
    pickle.dump(opp_partisans,open("data/opp_partisans.pickle","wb"))

In [9]:
print("Total: {} government partisans and {} opposition partisans were filtered".format(len(gov_partisans),len(opp_partisans)))

Total: 35422 government partisans and 26465 opposition partisans were filtered


In [10]:
print("Active in March: {} government partisans and {} opposition partisans".format(len(gov_ids), len(opp_ids)))

NameError: name 'gov_ids' is not defined

In [2]:
auth = AppAuthHandler(consumer_key, consumer_secret)
api = API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

In [3]:
scraper = FollowerScraper(api = api, path_log = "scraper.log")

In [ ]:
def intersect_followers(followers_dict):
    profile_names = list(followers_dict.keys())
    followers_intersections = [pd.Series(True, index=list(elements), name=name)
                      for name, elements in followers_dict.items()] #set all values to true where id exists
    followers_intersections = pd.concat(followers_intersections,sort = False,axis = 1) #concatanate
    followers_intersections = followers_intersections.fillna(False).reset_index() #fill nas and put id to columns
    followers_intersections = followers_intersections.groupby(profile_names).count() #get counts
    return followers_intersections["index"]

In [ ]:
opposition_followers = {k:set(v) for k,v in opposition_followers.items()}
setplot(intersect_followers(opposition_followers),show_percentages = True)
plt.show()

In [ ]:
government_followers = {k:set(v) for k,v in government_followers.items()}
government_intersections = intersect_followers(government_followers)
setplot(government_intersections,show_percentages = True)
plt.show()
print(pd.DataFrame(government_intersections).sort_values(by = "index"))

Population data:

In [3]:
cols = ['id_str',
 'followers_count',
 'created_at',
 'location',
 'statuses_count',
 'party',
 'screen_name',
 'verified',
 'status-created_at',             
 'profile_location',
 'profile_location-bounding_box',
 'profile_location-contained_within',
 'profile_location-country',
 'profile_location-country_code',
 'profile_location-full_name',
 'profile_location-id',
 'profile_location-name',
 'profile_location-place_type',
 'profile_location-url']

In [15]:
government_partisans = pd.read_csv("data/gov_partisans_info.csv",index_col = 0)
opposition_partisans = pd.read_csv("data/opp_partisans_info.csv",index_col = 0)
government_partisans["party"] = "government"
opposition_partisans["party"] = "opposition"
government_partisans = government_partisans#.filter(cols)
opposition_partisans = opposition_partisans#.filter(cols)
partisans = pd.concat([government_partisans,opposition_partisans], axis = 0)

/home/piotr/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (0,1,2,11,12,18,19,20,21,23,24,25,26,27,28,29,31,32,40,41,42,43,45,46,47,48,50,51,55,56,57,61,68,69,70,71,73,74,100,101,102,103,134,135,136,137,138,139,140,141,142,143,144,145) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/piotr/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (0,1,2,6,10,11,12,15,16,18,19,20,21,23,24,32,34,35,37,38,39,40,42,43,51,53,54,56,57,58,59,61,62,68,69,73,74,75,79,86,87,88,89,138,139,140,141,142,143,144) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [16]:
partisans.head(3)

,id,id_str,name,screen_name,location,profile_location-id,profile_location-url,profile_location-place_type,profile_location-name,profile_location-full_name,...,status-coordinates-coordinates,status-retweeted_status-scopes-followers,status-withheld_in_countries,withheld_in_countries,status-retweeted_status-geo-type,status-retweeted_status-geo-coordinates,status-retweeted_status-coordinates-type,status-retweeted_status-coordinates-coordinates,status-retweeted_status-withheld_in_countries,party
0,858408039895924736,858408039895924736,Marek,Marek02189962,"Płock, Polska",049c39650b58e84b,https://api.twitter.com/1.1/geo/id/049c39650b5...,unknown,"Płock, Polska","Płock, Polska",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,government
1,901537783654776832,901537783654776832,ryszard Olędzki,ryszard_oledzki,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,government
2,1241450557744742402,1241450557744742402,Collegium Humanum,CollegiumHuman2,"Warszawa, Polska",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,government


Remove corrupt cases that resulted from some errors in merging the data:

In [17]:
#coerce numeric to string and remove decimal 0 from floats
partisans[["id_str","statuses_count","followers_count"]] = partisans[["id_str","statuses_count","followers_count"]].applymap(lambda x: str(x).split('.')[0])
#get numeric mask 
numeric = partisans[["id_str","statuses_count","followers_count"]].applymap(lambda x: x.isnumeric())
#replace non-numeric with nas:
partisans[["id_str","statuses_count","followers_count"]] = partisans[["id_str","statuses_count","followers_count"]].where(numeric)

Print corrupt cases:

In [31]:
gov_partisans = pickle.load(open("data/gov_partisans.pickle","rb"))
list(gov_partisans).index(760361543410262017)+1
list(gov_partisans)[1864]
scraper = FollowerScraper(api = api, path_log = "temp.log")
good = api.get_user(760361543410262017)._json
bad = api.get_user(793949309783908357)._json
good_all = scraper._traverse_json(good,dict())
bad_all = scraper._traverse_json(bad,dict())
rows = []
import csv
with open("data/gov_partisans_info.csv","r") as f:
    reader = csv.reader(f)
    for i, row in enumerate(reader):
        if i in {1864,1865}:
            rows.append(row)
partisans.iloc[1863:1866]

In [8]:
nas = partisans[["id_str","statuses_count","followers_count"]].applymap(lambda x: pd.isna(x)).sum(axis = 1) > 0
partisans.loc[nas]

,id_str,followers_count,created_at,location,statuses_count,party,screen_name,verified,status-created_at,profile_location,profile_location-bounding_box,profile_location-contained_within,profile_location-country,profile_location-country_code,profile_location-full_name,profile_location-id,profile_location-name,profile_location-place_type,profile_location-url
1864,NaN,NaN,NaN,NaN,NaN,government,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1865,NaN,NaN,False,"[{'url': 'https://t.co/5SOMCrfPuc', 'expanded_...",NaN,government,[],NaN,https://abs.twimg.com/sticky/default_profile_i...,NaN,NaN,NaN,NaN,NaN,NaN,"<a href=""http://www.twitter.com"" rel=""nofollow...",NaN,NaN,NaN
6485,3334104364,NaN,NaN,NaN,NaN,government,steto_tomasz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6486,NaN,NaN,[],663,NaN,government,170,NaN,NaN,NaN,NaN,2349,False,False,NaN,3,NaN,2769,Fri Jun 19 05:06:55 +0000 2015
21154,4707795327,NaN,NaN,Wrocław,NaN,government,bozkagrabna,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21155,NaN,NaN,[],905,NaN,government,89,NaN,NaN,NaN,NaN,40,False,False,NaN,0,NaN,34,Mon Jan 04 19:55:50 +0000 2016
23569,731173113149464577,NaN,NaN,NaN,NaN,government,kameleon__73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23570,NaN,NaN,"[{'screen_name': 'bogdan607', 'name': 'Radosła...",312,NaN,government,183,NaN,NaN,1.24523e+18,NaN,1479,False,False,NaN,0,NaN,2141,Fri May 13 17:23:49 +0000 2016
34140,4165595057,NaN,NaN,NaN,NaN,government,misiek1962,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34141,NaN,NaN,"[{'screen_name': 'MichalGornicki', 'name': 'Mi...",227,NaN,government,81,NaN,NaN,7.93439e+17,NaN,2842,False,False,NaN,0,NaN,2535,Wed Nov 11 18:22:06 +0000 2015


In [594]:
partisans = partisans.dropna(subset = ["id_str","statuses_count","followers_count"])

In [595]:
partisans[["id_str","statuses_count","followers_count"]] = partisans[["id_str","statuses_count","followers_count"]].astype(np.int64)

Convert dates:

In [596]:
partisans.created_at = pd.to_datetime(partisans.created_at,format = '%a %b %d %H:%M:%S +0000 %Y')
partisans["status-created_at"] = pd.to_datetime(partisans["status-created_at"],format = '%a %b %d %H:%M:%S +0000 %Y')

Filter out users who tweeted since march 2020:

In [597]:
partisans = partisans[partisans["status-created_at"] >= datetime.datetime.strptime("03/2020","%m/%Y")]

Shuffle:

In [598]:
partisans = partisans.iloc[np.random.choice(np.arange(partisans.shape[0]), partisans.shape[0], replace = False)]

In [624]:
if os.path.isfile("data/gov_ids.pickle"):
    gov_ids = pickle.load(open("data/gov_ids.pickle","rb"))
else:
    gov_ids = partisans[partisans.party == "government"].id_str.astype("str").tolist()
    pickle.dump(gov_ids, open("data/gov_ids.pickle","wb"))
    
    
if os.path.isfile("data/opp_ids.pickle"):
    opp_ids = pickle.load(open("data/opp_ids.pickle","rb"))
else:
    opp_ids = partisans[partisans.party == "opposition"].id_str.astype("str").tolist()
    pickle.dump(opp_ids, open("data/opp_ids.pickle","wb"))

Most tweeting:

In [578]:
partisans.sort_values(by = "created_at", ascending = False)

,id_str,followers_count,created_at,location,statuses_count,party,screen_name,verified,status-created_at,profile_location,profile_location-bounding_box,profile_location-contained_within,profile_location-country,profile_location-country_code,profile_location-full_name,profile_location-id,profile_location-name,profile_location-place_type,profile_location-url
56205,1244652089453797382,3,2020-03-30 15:46:13,NaN,27,opposition,Andrzej07654467,False,2020-04-01 21:57:49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36171,1244648454414106629,2,2020-03-30 15:31:41,NaN,13,opposition,PanPIOTR13,False,2020-04-02 19:44:53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51817,1244642669932032000,2,2020-03-30 15:08:43,NaN,1,opposition,Stefani77739340,False,2020-03-30 15:14:22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53208,1244642370844594178,0,2020-03-30 15:07:28,NaN,36,opposition,L0rdDr,False,2020-04-02 16:59:29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43354,1244627292988608518,1,2020-03-30 14:07:43,Warszawa,10,opposition,JarekKuzniarek,False,2020-03-31 11:22:31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35719,7341922,89829,2007-07-09 07:21:11,Warszawa,64399,opposition,Wimmer,False,2020-03-30 12:06:04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54896,6521392,395,2007-06-02 12:10:11,"Warsaw, Poland",1115,opposition,kgoworek,False,2020-03-28 15:33:04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48810,3082781,21,2007-03-31 21:49:45,NaN,3710,opposition,ds3233,False,2020-04-02 14:44:36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4606,3033611,118,2007-03-31 02:33:47,"Wroclaw, Poland",692,government,DonKoyote,False,2020-03-29 16:08:05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
